In [1]:
import moose
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import plotly.graph_objects as go

In [2]:
container=moose.Neutral('/sim')
model=moose.Neutral(f'{container.path}/model')
data=moose.Neutral(f'{container.path}/data')
axon=moose.Compartment(f'{model.path}/axon')

In [3]:
kchan=moose.HHChannel(f'{axon.path}/K')
moose.connect(kchan,'channel',axon,'channel')

<moose.SingleMsg id=5 dataIndex=0 path=/Msgs[0]/singleMsg[0]>

In [4]:
axon.Em=0
axon.initVm=0
axon.Cm=1
axon.Rm=1/0.3

In [5]:
# Setting up the K channel

kchan.Ek=-12
kchan.Gbar=36
kchan.Xpower=4

n_gate=moose.HHGate(f'{kchan.path}/gateX')
n_alpha_params=[0.1, -0.01, -1.0, -10.0, -10.0]
n_beta_params=[0.125, 0, 0, 0, 80.0]

vdivs=150
vmin=-30
vmax=120

n_params=n_alpha_params+n_beta_params+[vdivs,vmin,vmax]
n_gate.setupAlpha(n_params)

True

In [6]:
# Setting up the Na gate

nachan=moose.HHChannel(f'{axon.path}/Na')
moose.connect(nachan,'channel',axon,'channel')

nachan.Ek=115
nachan.Gbar=120
nachan.Xpower=3
nachan.Ypower=1

m_gate=moose.HHGate(f'{nachan.path}/gateX')
h_gate=moose.HHGate(f'{nachan.path}/gateY')

m_alpha_params=[2.5, -0.1, -1.0, -25.0, -10.0]
m_beta_params=[4, 0, 0, 0, 18.0]

h_alpha_params=[0.07, 0, 0, 0, 20.0]
h_beta_params=[1, 0, 1, -30, -10.0]


m_params=m_alpha_params+m_beta_params+[vdivs,vmin,vmax]
h_params=h_alpha_params+h_beta_params+[vdivs,vmin,vmax]
m_gate.setupAlpha(m_params)
h_gate.setupAlpha(h_params)

True

In [7]:

pulse=moose.PulseGen(f'{model.path}/pulse')
moose.connect(pulse,'output',axon,'injectMsg')

<moose.SingleMsg id=5 dataIndex=2 path=/Msgs[0]/singleMsg[2]>

In [8]:
pulse.baseLevel=0
pulse.width[0]=90
pulse.delay[0]=10

In [9]:
gK_tab=moose.Table(f'{data.path}/K')
gNa_tab=moose.Table(f'{data.path}/Na')
Vm_tab=moose.Table(f'{data.path}/Vm')
moose.connect(gK_tab,'requestOut',kchan,'getGk')
moose.connect(gNa_tab,'requestOut',nachan,'getGk')
moose.connect(Vm_tab,'requestOut',axon,'getVm')

<moose.SingleMsg id=5 dataIndex=5 path=/Msgs[0]/singleMsg[5]>

In [10]:
n_tab=moose.Table(f'{data.path}/n_particle')
moose.connect(n_tab,'requestOut',kchan,'getX')
m_tab=moose.Table(f'{data.path}/m_particle')
moose.connect(m_tab,'requestOut',nachan,'getX')
h_tab=moose.Table(f'{data.path}/h_particle')
moose.connect(h_tab,'requestOut',nachan,'getY')

<moose.SingleMsg id=5 dataIndex=8 path=/Msgs[0]/singleMsg[8]>

In [11]:
def run_simulation(I_inj):
    pulse.level[0]=I_inj
    simtime=110
    moose.reinit()
    moose.start(simtime)

    globals()['t']=np.arange(len(Vm_tab.vector))*Vm_tab.dt